In [1]:
# Import packages and load data
import pandas as pd
import glob
import re


In [2]:
# Import data into dataframe
def load_data():
    files = glob.glob("../data/raw" + "/*.csv")
    return pd.concat([pd.read_csv(filename, index_col=None, header=0) for filename in files], axis=0, ignore_index=True)

In [3]:
### Combine individual Asset Volume Columns ###
def create_volume_asset(df):
    if not "volume_asset" in df:
        df["volume_asset"] = 0
        rexp = r'Volume\s(?!USDT)'
        for col in df.columns:
            if re.search(rexp, col) is not None:
                new_col = pd.DataFrame({"volume_asset": df[col]})
                df.update(new_col)
                df = df.drop(columns=[col])
    return df

In [5]:
data = (
    load_data()
    .rename(columns={"Volume USDT": "volume_usdt", "tradecount": "trade_count"})
    .pipe(create_volume_asset)
)
data = data.assign(
    date = lambda x: pd.to_datetime(x["date"]),
    asset_coin = lambda x: [symbol.split("/")[0] for symbol in x["symbol"]],
    base_coin = lambda x: [symbol.split("/")[1] for symbol in x["symbol"]],
    avg_trade_asset = lambda x: x["volume_asset"] / x["trade_count"],
    avg_trade_usdt = lambda x: x["volume_usdt"] / x["trade_count"],
    delta_value = lambda x: (x["close"] - x["open"]) / x["open"],
    delta_range = lambda x: x["high"] - x["low"]
)
data = (
    data
    .drop(columns=["symbol", "unix", "open", "high", "low", "close"])
)
data

,date,volume_usdt,trade_count,volume_asset,asset_coin,base_coin,avg_trade_asset,avg_trade_usdt,delta_value,delta_range
0,2021-01-13,8.631707e+03,90.0,2.451996e+07,BTT,USDT,272444.022222,95.907850,-0.001134,0.000003
1,2021-01-12,3.011956e+06,20031.0,8.455808e+09,BTT,USDT,422136.113225,150.364721,0.005990,0.000036
2,2021-01-11,4.024436e+06,25602.0,1.139497e+10,BTT,USDT,445081.251621,157.192270,-0.185797,0.000117
3,2021-01-10,7.137798e+06,41729.0,1.744114e+10,BTT,USDT,417962.006854,171.051263,0.047375,0.000088
4,2021-01-09,2.381440e+07,105684.0,5.561435e+10,BTT,USDT,526232.435752,225.335876,0.234549,0.000203
...,...,...,...,...,...,...,...,...,...,...
16137,2017-12-18,5.392264e+06,NaN,1.703670e+04,LTC,USDT,NaN,NaN,0.130379,83.960000
16138,2017-12-17,4.534011e+06,NaN,1.441388e+04,LTC,USDT,NaN,NaN,0.059292,38.800000
16139,2017-12-16,2.613906e+06,NaN,8.895820e+03,LTC,USDT,NaN,NaN,-0.000102,20.000000
16140,2017-12-15,4.569740e+06,NaN,1.624413e+04,LTC,USDT,NaN,NaN,0.079295,74.220000
